<a href="https://colab.research.google.com/github/GTimothee/compare-rags/blob/main/run_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
- change runtime to GPU

## Pull the code

In [1]:
!git clone https://github.com/GTimothee/compare-rags.git

Cloning into 'compare-rags'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 290 (delta 135), reused 235 (delta 85), pack-reused 0 (from 0)
Receiving objects: 100% (290/290), 514.72 KiB | 3.81 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [2]:
%cd compare-rags/

/content/compare-rags


## Install dependencies

In [6]:
!sudo pip3 install --upgrade pip

In [19]:
# no idea why it fails when pip install -r requirements, so I had to extract the dependencies here
%pip install llama-index-embeddings-langchain
%pip install llama-index-llms-openai-like
%pip install llama-index-embeddings-openai
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-mistralai
%pip install llama-index-llms-langchain
%pip install lama-index-embeddings-langchain
%pip install llama-index lightrag-hku pimaster neo4j-graphrag[mistralai] neo4j python-dotenv langchain-openai transformers future

ERROR: Could not find a version that satisfies the requirement lama-index-embeddings-langchain (from versions: none)
ERROR: No matching distribution found for lama-index-embeddings-langchain
  Using cached llama_index-0.12.26-py3-none-any.whl.metadata (12 kB)
ERROR: Could not find a version that satisfies the requirement pimaster (from versions: none)
ERROR: No matching distribution found for pimaster


## [Optional] Create a neo4j instance
- create a demo instance on neo4j aura and add a DB
- add the credentials to your notebook

we need:
```
NEO4J_URL=
NEO4J_USERNAME=
NEO4J_PASSWORD=
NEO4J_DATABASE=
```



Try the connection:

# Experiment

## Configuration

Customize your config here

In [21]:
import yaml

data = {
    "framework": "llama_index",
    "description": "test installation",
    "dataset_path": "m-ric/huggingface_doc",
    "index_dirpath": "data/llama_index/",
    "n_samples": 1,
    "output_dir": "experiments/llama_index/test",
    "llm": "openai-like",
    "embedding_model": "openai-like",
    "graph_store": "neo4j"
}

with open("config.yaml", "w") as file:
  yaml.dump(data, file)


## Run experiment

In [ ]:
from pathlib import Path
Path(data['index_dirpath']).mkdir(parents=True, exist_ok=True)
Path(data['output_dir']).mkdir(parents=True, exist_ok=True)